## Langgraph

# Lecture 11- Langgraph Day 4

Simple tool call with Langgraph must for stable tasks

### Step 1: Setup LLM code

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()  # Load environment variables from .env file

# Corrected way to retrieve the API key
api_key = os.getenv("GOOGLE_API_KEY")

if not api_key:
    raise ValueError("GOOGLE_API_KEY not found. Make sure it's set in the .env file.")

os.environ["GOOGLE_API_KEY"] = api_key

from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest")

In [ ]:
# Step 1: Define a tool

from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from tavily import TavilyClient

wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=1000000))
tavily_client = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))


In [ ]:
query = "Indian Food"

In [ ]:
from langchain_core.messages import AIMessage
from langchain_core.tools import tool

from langgraph.prebuild import ToolNode

In [ ]:
@tool
def get_wikipedia_result(query: str):
    "Call to get wikipedia result"
    wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=1000000))
    return wikipedia.run({"query": query})

@tool
def get_tavily_results(query: str):
    "Call to get tavily results"
    response = tavily_client.search(query, max_results=10)
    content = ""
    for result in response['results']:
        content += result['content']
    return content

In [ ]:
tool1 = [get_wikipedia_result]
tool2 = [get_tavily_results]
tool_node1 = ToolNode(tool1)
tool_node2 = ToolNode(tool2)

In [ ]:
message_with_single_tool_call_wiki = AIMessage(
    content=""
    tool_calls=[
        {
            "name": "get_wikipedia_result",
            "args": {"query": query},
            "id": "tool_call_1",
            "type": "tool_call",
        }
    ])

response_tool_1 = tool_node1.invoke({"messages": [message_with_single_tool_call_wiki]})

In [ ]:
for m in response_tool_1['messages']:
    m.pretty_print()

In [ ]:
message_with_single_tool_call_tavily = AIMessage(
    content=""
    tool_calls=[
        {
            "name": "get_tavily_results",
            "args": {"query": query},
            "id": "tool_call_id",
            "type": "tool_call",
        }
    ])

response_tool_2 = tool_node2.invoke({"messages": [message_with_single_tool_call_tavily]})

In [ ]:
for m in response_tool_2['messages']:
    m.pretty_print()

In [ ]:
system_message_summarize = f'''You are expert at a giving a detailed summary of the content given to you.
User Query: {query}

Wikipedia Result: {response_tool_1['messages']}

Tavily Result: {response_tool_2['messages']}

Always attibute the responses to sources (Wikipedia and Tavily)
'''

In [ ]:
final_summary = llm.invoke(system_message_summarize)

In [ ]:
print(final_summary.content)

In [ ]:
with open("results.txt", "w") as file:
    file.write(final_summary.content)